In [1]:
from copy import deepcopy

In [6]:
Roraima = {
    'Uiramuta': ['Pacaraima', 'Normandia'],
    'Pacaraima': ['Uiramuta', 'Normandia', 'Boa Vista', 'Amajari'],
    'Normandia': ['Uiramuta', 'Bonfim', 'Boa Vista', 'Pacaraima'],
    'Boa Vista': ['Pacaraima', 'Normandia', 'Bonfim', 'Canta', 
                  'Mucajai', 'Alto Alegre', 'Amajari'],
    'Bonfim': ['Caracarai', 'Canta', 'Boa Vista', 'Normandia'],
    'Canta': ['Bonfim', 'Caracarai', 'Iracema', 'Mucajai', 'Boa Vista'],
    'Amajari': ['Pacaraima', 'Boa Vista', 'Alto Alegre'],
    'Alto Alegre': ['Amajari', 'Boa Vista', 'Mucajai', 'Iracema'],
    'Mucajai': ['Alto Alegre', 'Boa Vista', 'Canta', 'Iracema'],
    'Iracema': ['Alto Alegre', 'Mucajai', 'Canta', 'Caracarai'],
    'Caracarai': ['Iracema', 'Canta', 'Bonfim', 'Caroebe', 'Sao Joao da Baliza', 
                  'Sao Luiz do Anaua', 'Rorainopolis'],
    'Caroebe': ['Sao Joao da Baliza', 'Caracarai'],
    'Sao Joao da Baliza': ['Rorainopolis', 'Sao Luiz do Anaua', 'Caracarai', 'Caroebe'],
    'Rorainopolis': ['Caracarai', 'Sao Luiz do Anaua', 'Sao Joao da Baliza'],
    'Sao Luiz do Anaua': ['Caracarai', 'Sao Joao da Baliza', 'Rorainopolis'],
}

class Problema:
    def __init__(self, bordas, cores):
        # bordas é um dicionário onde cada chave é uma região
        # e seu valor é uma lista de suas regiões fronteiriças
        self.bordas = bordas

        # cores é uma lista com as 4 cores a serem usadas na coloração
        self.cores = cores

class No:
    def __init__(self, mapa, regiao, cor):
        # mapa é um dicionario
        # cada chave é uma região, seu valor é sua cor atual
        # ex: {'São Paulo': 'azul', 'Paraíba': 'vermelho'}
        # representa diferentes estados do sistema
        self.mapa = deepcopy(mapa)

        # ao criar um novo nó, passando uma região e uma cor,
        # o nó é criado com a coloração atualizada
        self.mapa[regiao] = cor

    def __str__(self):
        string = ''
        for regiao in self.mapa:
            if self.mapa[regiao] != '':
                string = string + f'{regiao}: {self.mapa[regiao]}\n'
        return string

    def __repr__(self):
        return self.__str__()
    
    # Ramifica o nó, método a ser chamado na busca
    # retorna uma lista com os filhos do nó
    def filhos(self, problema, proxima_regiao):
        filhos = []
        for cor in problema.cores:
            filho = No(self.mapa, proxima_regiao, cor)
            filhos.append(filho)
        return filhos
    
    # varre o mapa, verifica se há regiões fronteiriças com a mesma cor
    # será usado para podar nós com estados inviáveis
    def is_viavel(problema):
        pass

    # varre o mapa, verifica se todas as regiões estão preenchidas
    def is_objetivo():
        pass

# classe para realizar a busca
class Coloracao:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = [] 
        #  fronteira do problema de busca

        self.status = 'Coloração iniciando'
        # status possíveis: 'Coloração iniciando', 'Coloração em andamento', 
        # 'Coloração finalizada com sucesso', 'Coloração sem sucesso'

        # mapa é um dicionário com todas as regiões e suas cores
        # começa em branco
        # representa o estado inicial do sistema
        self.mapa = {}
        for regiao in self.problema.bordas:
            self.mapa[regiao] = ''

    def passo(self):
        if self.status == 'Coloração iniciando':
            primeira_região = list(self.mapa.keys())[0]
            # pegamos a primeira região do dicionário, que está todo em branco

            # criaremos 4 nós, para as 4 possíveis cores para a primeira região
            for cor in self.problema.cores:
                novo_no = No(self.mapa, primeira_região, cor)
                self.fronteira.append(novo_no)

            return

        if self.status == 'Coloração finalizada':
            print('Coloração finalizada')
            return
        
        try:
            proximo_no = self.fronteira.pop(-1)
        except IndexError:
            self.situacao = 'Coloração sem sucesso'
            return

        proxima_regiao = next(regiao for regiao in proximo_no.mapa if proximo_no.mapa[regiao] == '')
        # se der erro, é pq o mapa está cheio

        for filho in proximo_no.filhos(self.problema, proxima_regiao):
            # fazer teste de bordas antes do append
            # 
            # tem que ser um método dentro de nó, que retorna true or false
            # if filho.teste()
            # se passar no teste, append o filho
            self.fronteira.append(filho)
        
        self.indice += 1

In [3]:
list(Roraima.keys())[1]

'Pacaraima'

In [7]:
prob_ex = Problema(Roraima, ['branco', 'verde', 'azul', 'cinza'])
col_ex = Coloracao(prob_ex)
no_1 = No(col_ex.mapa, 'Uiramuta', 'branco')

In [25]:
try:
    proxima_regiao = next(regiao for regiao in no_1.mapa if no_1.mapa[regiao] == '')
except StopIteration:
    proxima_regiao = "ops"

proxima_regiao

'ops'

In [19]:
col_ex.problema.cores

['branco', 'verde', 'azul', 'cinza']

In [11]:
filhos = no_1.filhos(prob_ex)
filhos

[Uiramuta: branco
 Pacaraima: branco,
 Uiramuta: branco
 Pacaraima: verde,
 Uiramuta: branco
 Pacaraima: azul,
 Uiramuta: branco
 Pacaraima: cinza]

In [15]:
no_1

Uiramuta: branco

In [18]:
proxima_regiao = next(regiao for regiao in no_1.mapa if no_1.mapa[regiao] == 'aa')
proxima_regiao

StopIteration: 

In [14]:
netos = []
for filho in filhos:
    novos_netos = filho.filhos(prob_ex)
    for neto in novos_netos:
        netos.append(neto)
# netos